In [1]:
import __init__
#
from taxi_common.file_handling_functions import load_pickle_file
from taxi_common.sg_grid_zone import get_sg_grid_xy_points, get_sg_zones
#
from IPython.display import HTML, display
import folium

from community_analysis import ss_trips_dpath, ss_trips_prefix
import pandas as pd
from taxi_common import sg_grid_geojson

In [2]:
x_points, y_points = get_sg_grid_xy_points()
len(x_points), len(y_points)

(96, 53)

In [4]:
x_points, y_points = get_sg_grid_xy_points()
xc, yc = (x_points[0] + x_points[-1]) / float(2), (y_points[0] + y_points[-1]) / float(2)
#
map_osm = folium.Map(location=[yc, xc], zoom_start=11)
for x in x_points:
    sx, sy, ex, ey = x, y_points[0], x, y_points[-1]
    map_osm.add_children(folium.PolyLine(locations=[(sy, sx), (ey, ex)], weight=1.5))
for y in y_points:
    sx, sy, ex, ey = x_points[0], y, x_points[-1], y
    map_osm.add_children(folium.PolyLine(locations=[(sy, sx), (ey, ex)], weight=1.5))
map_osm.save('/Users/JerryHan88/Desktop/zonePartition.html')
map_osm

In [14]:
def draw_zones(df, withHeatMap = False):
    x_points, y_points = get_sg_grid_xy_points()
    xc, yc = (x_points[0] + x_points[-1]) / float(2), (y_points[0] + y_points[-1]) / float(2)
    #
    map_osm = folium.Map(location=[yc, xc], zoom_start=11)
    for x in x_points:
        sx, sy, ex, ey = x, y_points[0], x, y_points[-1]
        map_osm.add_children(folium.PolyLine(locations=[(sy, sx), (ey, ex)], weight=1.0))
    for y in y_points:
        sx, sy, ex, ey = x_points[0], y, x_points[-1], y
        map_osm.add_children(folium.PolyLine(locations=[(sy, sx), (ey, ex)], weight=1.0))
    #
    zones = get_sg_zones()
    allZones = set(['%d#%d' % (zi, zj) for zi, zj in zones.iterkeys()])
    df['zizj'] = df.apply(lambda x: '%d#%d' % (x['zi'], x['zj']), axis=1)
    tripOccuredZones = set(df['zizj'])
    zizj_tripNum_df = df.groupby(['zizj']).count()['did'].to_frame('numTrips').reset_index()
    if withHeatMap:
        noneTripZones = allZones.difference(tripOccuredZones)
        for zizj in noneTripZones:
            zizj_tripNum_df.loc[-1] = [zizj, 0]
            zizj_tripNum_df.index = zizj_tripNum_df.index + 1
        map_osm.geo_json(geo_path=sg_grid_geojson, data=zizj_tripNum_df,
                    columns=['zizj', 'numTrips'],
                    key_on='feature.id',
                    fill_color='BuPu', fill_opacity=0.4, line_opacity=0.2)
    return map_osm

In [9]:
df = pd.read_csv('%s/%s2009.csv' % (ss_trips_dpath, ss_trips_prefix))

In [15]:
map_osm = draw_zones(df, withHeatMap = True)

In [17]:
map_osm.save('/Users/JerryHan88/Desktop/zonePartition.html')